# Introduction

[Label Efficient Learning of Transferable Representations Across Domains and Tasks](6621-label-efficient-learning-of-transferable-representations-acrosss-domains-and-tasks.pdf)


## TLDR;


## NOTES

### TODO
1. Process the MNIST and SVNH Dataset as required:  Done
  * MNIST : 5-9 Classes - Done (Target Domain)
  * SVNH : 0-4 Classes - Done (Source Domain)
2. Creation of Architecture as shown in the image : ![architecture](resources/Architecture.png) - Keras Issue - Need Debugging
3. Train the Model with Optimiser Specified Separately and load the weights in the main experiment.
4. Initialise Weights only for Source CNN Model with the pretrained weights.
5. Proceed with Trainning the Entire Architecture.



# Six different methods : 
* (i) Target only: the model is trained on D2 from scratch; 
* (ii) Fine-tune: the model is pretrained on D1 and fine-tuned on D2;
* (iii) Matching networks [70]: we first pretrain the model on D3, then use D2 as the support set in the matching networks; 
* (iv) Fine-tuned matching networks: same as baseline iii, except that for each k the model is fine-tuned on D2 with 5-way (k 􀀀 1)-shot learning: k 􀀀 1 examples in each class are randomly selected as the support set, and the last example in each class is used as the query set; 
* (v) Fine-tune + adversarial: in addition to baseline ii, the model is also trained on D1 and D3 with a domain adversarial loss; 
* (vi.) Full model: fine-tune the model with the proposed multi-layer domain adversarial loss.




# Module : # TO DO:
Softmax with Temperature

### Imports

In [2]:
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
from torch.tensor import Tensor as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision.datasets import MNIST,SVHN

print(torch.__version__) # torch Version


1.4.0


##### Hyperparams:


In [ ]:


# Hyperparams:
bs=64
shuffle=True

###### Dataset and Dataloaders

In [3]:
# To Do:
# Filter the Dataset Based on Range of Labels Given
class CustomMNIST(MNIST):
    """
    Additional Args:
        min_label : Minimun Value of Label That is to be allowed. 
        # Todo : Need to support list of values instead of min value
    """
    def __init__(self, root, train=True, transform=None, target_transform=None, download=False,min_label=5):
        super().__init__(root, train=train, transform=transform, target_transform=target_transform, download=download)
        self.min_label=min_label
        if train:
            self.train_data=self.train_data[self.train_labels>=self.min_label]
            self.train_labels=self.train_labels[self.train_labels>=self.min_label]

class CustomSVNH(SVHN):
    def __init__(self, root, split='train', transform=None, target_transform=None, download=False,max_label=5):
        super().__init__(root, split=split, transform=transform, target_transform=target_transform, download=download)
        self.max_label=max_label
        if split=='train':
            self.data=self.data[self.labels<=self.max_label]
            self.labels=self.labels[self.labels<=self.max_label]

svnh_ds=CustomSVNH('.',max_label=4)
mnist_ds=CustomMNIST('.',min_label=5)
mnist_dl=DataLoader(mnist_ds,batch_size=bs,shuffle=shuffle)
svnh_dl=DataLoader(svnh_ds,batch_size=bs,shuffle=shuffle)


RuntimeError: Dataset not found or corrupted. You can use download=True to download it

In [ ]:
# Checked And working as expected
# mnist=FilteredMnist('.',download=True,label_min=5)
# print((mnist.train_labels))
# svnh=CustomSVNH(root='.',split='train',download=True,max_label=4)
# mnist=CustomMNIST(root='.',train=True,download=True,min_label=5)
# print(svnh.data.shape,svnh.labels.shape)
# #print(np.unique(svnh.labels))
# #print(np.unique(mnist.train_labels))
# print(mnist.train_data.shape,mnist.train_labels.shape)


In [ ]:
#  MNIST : 5-9 Classes -  (Target Domain)
#  SVNH : 0-4 Classes -  (Source Domain)

svnh_ds=CustomSVNH('.',max_label=4)
mnist_ds=CustomMNIST('.',min_label=5)
bs=64
mnist_dl=DataLoader(mnist_ds,batch_size=bs,shuffle=True)
svnh_dl=DataLoader(svnh_ds,batch_size=bs,shuffle=True)




#### Utils


In [ ]:
import numpy as np 
import torch
import torch.nn.functional as F


# def softmax_temperature(logits, tau=1.0, hard=False, eps=1e-10):
#     # Use inbuilt Torch Functional Version of Smoothened Softmax
#     return F.gumbel_softmax(logits=logits, tau=tau, hard=hard, eps=eps)  


def softmax_temperature(out, temperature=1.0):
    # helper function to sample an index from a probability array
    out = np.log(out) / temperature
    out = np.exp(out) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, out, 1))

def distillation(y, teacher_scores, labels, T, alpha):
    p = F.log_softmax(y/T, dim=1)
    q = F.softmax(teacher_scores/T, dim=1)
    l_kl = F.kl_div(p, q, size_average=False) * (T**2) / y.shape[0]
    l_ce = F.cross_entropy(y, labels)
    return l_kl * alpha + l_ce * (1. - alpha)


#### Modelling

In [ ]:


# To Do Finalise on the Architecture

class MyNetwork(nn.Module):
    
    def __init__():
        super().__init__()
        # self.conv1=nn.Conv2d()
        pass
